In [1]:
import bilby
from copy import deepcopy
import numpy as np

In [11]:
injection_parameters = dict(
# Mass parameters
    mass_1=1.5,
    mass_2=1.3,

# Spin parameters
    chi_1=0.02,
    chi_2=0.02,

# Distance parameter
    luminosity_distance=50.0,

# Angular orientation parameters
    theta_jn=0.4,
    psi=2.659,
    phase=1.3,

# Time parameter
    geocent_time=1126259642.413,

# Sky Location
    ra=1.375,
    dec=-1.2108,

# Tidal Deformability parameters
    lambda_1=545.0,
    lambda_2=1346.0,

# Serves as a reference marker for the injection parameters.
    fiducial=1, # Used while reweighting to identify the injection parameters
)
injection_parameters["a_1"] = injection_parameters["chi_1"]
injection_parameters["a_2"] = injection_parameters["chi_2"]

injection_parameters

{'mass_1': 1.5,
 'mass_2': 1.3,
 'chi_1': 0.02,
 'chi_2': 0.02,
 'luminosity_distance': 50.0,
 'theta_jn': 0.4,
 'psi': 2.659,
 'phase': 1.3,
 'geocent_time': 1126259642.413,
 'ra': 1.375,
 'dec': -1.2108,
 'lambda_1': 545.0,
 'lambda_2': 1346.0,
 'fiducial': 1,
 'a_1': 0.02,
 'a_2': 0.02}

In [3]:
################################ WAVEFORM MODEL ################################


# Set the duration and sampling frequency of the data segment that we're
# going to inject the signal into
duration = 32
sampling_frequency = 2048.0
minimum_frequency = 40.0
start_time = injection_parameters["geocent_time"] + 2 - duration

# Fixed arguments passed into the source model
waveform_arguments = dict(
    waveform_approximant="IMRPhenomPv2_NRTidal", # A phenomenological 
                         # Inspiral-Merger-Ringdown model for a BNS
    reference_frequency=50.0,
    minimum_frequency=minimum_frequency,
)

# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    # frequency_domain_source_model=bilby.gw.source.lal_binary_neutron_star,
    frequency_domain_source_model=bilby.gw.source.lal_binary_neutron_star_relative_binning,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters,
    waveform_arguments=waveform_arguments,
)


13:48 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star_relative_binning
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters


In [4]:
################################### DETECTOR ###################################


# Set up interferometers.  In this case we'll use three interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1) and Virgo (V1)).
# These default to their design sensitivity
ifos = bilby.gw.detector.InterferometerList(["H1", "L1", "V1"])
for interferometer in ifos:
    interferometer.minimum_frequency = 40
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=start_time,
)
ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
)


/home/kay/anaconda3/envs/thesis/lib/python3.11/site-packages/lalsimulation/lalsimulation.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal
13:48 bilby INFO    : Injected signal in H1:
13:48 bilby INFO    :   optimal SNR = 34.44
13:48 bilby INFO    :   matched filter SNR = 34.58+0.89j
13:48 bilby INFO    :   mass_1 = 1.5
13:48 bilby INFO    :   mass_2 = 1.3
13:48 bilby INFO    :   chi_1 = 0.02
13:48 bilby INFO    :   chi_2 = 0.02
13:48 bilby INFO    :   luminos

[{'plus': array([ 0.00000000e+00-0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00-0.00000000e+00j, ...,
         -4.23008636e-25+7.98210938e-26j, -4.23011798e-25+7.97089719e-26j,
          0.00000000e+00-0.00000000e+00j]),
  'cross': array([0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         7.95519924e-26+4.21582544e-25j, 7.94402485e-26+4.21585695e-25j,
         0.00000000e+00+0.00000000e+00j])},
 {'plus': array([ 0.00000000e+00-0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00-0.00000000e+00j, ...,
         -4.23008636e-25+7.98210938e-26j, -4.23011798e-25+7.97089719e-26j,
          0.00000000e+00-0.00000000e+00j]),
  'cross': array([0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         7.95519924e-26+4.21582544e-25j, 7.94402485e-26+4.21585695e-25j,
         0.00000000e+00+0.00000000e+00j])},
 {'plus': ar

In [5]:
priors = bilby.gw.prior.BNSPriorDict()
for key in [
    "psi",
    "geocent_time",
    "ra",
    "dec",
    "chi_1",
    "chi_2",
    "theta_jn",
    "phase",
]:
    priors[key] = injection_parameters[key]
priors

13:48 bilby INFO    : No prior given, using default BNS priors in /home/kay/anaconda3/envs/thesis/lib/python3.11/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


{'mass_1': Constraint(minimum=0.5, maximum=5, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=0.5, maximum=5, name='mass_2', latex_label='$m_2$', unit=None),
 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False),
 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=0.4, maximum=4.4, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': -1.2108,
 'ra': 1.375,
 'theta_jn': 0.4,
 'psi': 2.659,
 'phase': 1.3,
 'chi_1': 0.02,
 'chi_2': 0.02,
 'lambda_1': Uniform(minimum=0, maximum=5000, name='lambda_1', latex_label='$\\Lambda_1$', unit=None, boundary=None),
 'lambda_2': Uniform(minimum=0, maximum=5000, name='lambda_2', lat

In [6]:
del priors["lambda_1"], priors["lambda_2"], priors["mass_ratio"]

priors["chirp_mass"] = bilby.core.prior.Gaussian(
    1.215, 0.1, name="chirp_mass", unit="$M_{\\odot}$"
)
priors["symmetric_mass_ratio"] = bilby.core.prior.Uniform(
    0.1, 0.25, name="symmetric_mass_ratio"
)
priors["lambda_tilde"] = bilby.core.prior.Uniform(0, 5000, name="lambda_tilde"
)
priors["delta_lambda_tilde"] = bilby.core.prior.Uniform(
    -500, 1000, name="delta_lambda_tilde"
)

# constraining these parameters to prevent out of bounds sampling.
priors["lambda_tilde"] = bilby.core.prior.Constraint(
    name="lambda_tilde", minimum=0, maximum=5000
)
priors["delta_lambda_tilde"] = bilby.core.prior.Constraint(
    name="delta_lambda_tilde", minimum=-500, maximum=1000
)
priors["lambda_1"] = bilby.core.prior.Constraint(
    name="lambda_1", minimum=100, maximum=5000
)
priors["lambda_2"] = bilby.core.prior.Constraint(
    name="lambda_2", minimum=100, maximum=5000
)

priors

{'mass_1': Constraint(minimum=0.5, maximum=5, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=0.5, maximum=5, name='mass_2', latex_label='$m_2$', unit=None),
 'chirp_mass': Gaussian(mu=1.215, sigma=0.1, name='chirp_mass', latex_label='$\\mathcal{M}$', unit='$M_{\\odot}$', boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': -1.2108,
 'ra': 1.375,
 'theta_jn': 0.4,
 'psi': 2.659,
 'phase': 1.3,
 'chi_1': 0.02,
 'chi_2': 0.02,
 'geocent_time': 1126259642.413,
 'symmetric_mass_ratio': Uniform(minimum=0.1, maximum=0.25, name='symmetric_mass_ratio', latex_label='$\\eta$', unit=None, boundary=None),
 'lambda_tilde': Constraint(minimum=0, maximum=5000, name='lambda_tilde', latex_label='$\\tilde{\\Lambda}$', unit=None),
 'delta_lambda_tilde': Constraint(minimum=-500, maximum=1000, name='delta_lambda_tilde', 

In [7]:
fiducial_parameters = injection_parameters.copy()
m1 = fiducial_parameters.pop("mass_1")
m2 = fiducial_parameters.pop("mass_2")
fiducial_parameters["chirp_mass"] = bilby.gw.conversion.component_masses_to_chirp_mass(
    m1, m2) # Chirp mass (M) (one of the prameters being estimated)
fiducial_parameters["mass_ratio"] = m2 / m1

l1=fiducial_parameters.pop("lambda_1")
l2=fiducial_parameters.pop("lambda_2")

fiducial_parameters["lambda_tilde"] =bilby.gw.conversion.lambda_1_lambda_2_to_lambda_tilde(
    l1,l2,m1,m2)
fiducial_parameters["delta_lambda_tilde"]=bilby.gw.conversion.lambda_1_lambda_2_to_delta_lambda_tilde(
   l1,l2,m1,m2)


fiducial_parameters

{'chi_1': 0.02,
 'chi_2': 0.02,
 'luminosity_distance': 50.0,
 'theta_jn': 0.4,
 'psi': 2.659,
 'phase': 1.3,
 'geocent_time': 1126259642.413,
 'ra': 1.375,
 'dec': -1.2108,
 'fiducial': 1,
 'chirp_mass': 1.2150360414642816,
 'mass_ratio': 0.8666666666666667,
 'lambda_tilde': 867.9931562541493,
 'delta_lambda_tilde': 95.05130053992274}

In [8]:
likelihood = bilby.gw.likelihood.RelativeBinningGravitationalWaveTransient(
    interferometers=ifos,
    waveform_generator=waveform_generator,
    priors=priors,
    distance_marginalization=True,
    fiducial_parameters=fiducial_parameters,
)

13:48 bilby INFO    : Loaded distance marginalisation lookup table from .distance_marginalization_lookup.npz.
13:48 bilby INFO    : Initial fiducial waveforms set up
13:48 bilby INFO    : Summary Data Obtained
13:48 bilby INFO    : Fiducial likelihood: 1052.32


In [9]:
reference_waveform = waveform_generator.frequency_domain_strain(fiducial_parameters)
print("Reference waveform:", reference_waveform)


13:48 bilby WARNING : There are unused waveform kwargs. This is deprecated behavior and will
result in an error in future releases. Make sure all of the waveform kwargs are correctly
spelled.

Unused waveform_kwargs: {'frequency_bin_edges': array([  40.     ,   41.4375 ,   42.96875,   44.625  ,   46.4375 ,
         48.4375 ,   50.65625,   53.09375,   55.84375,   58.90625,
         62.34375,   66.25   ,   70.71875,   75.84375,   81.75   ,
         88.65625,   96.78125,  106.3125 ,  117.5625 ,  130.8125 ,
        146.3125 ,  164.25   ,  184.65625,  207.3125 ,  231.9375 ,
        258.     ,  285.09375,  312.75   ,  340.625  ,  368.4375 ,
        396.03125,  423.25   ,  450.     ,  476.28125,  502.     ,
        527.21875,  551.90625,  576.03125,  599.6875 ,  622.84375,
        645.5    ,  667.71875,  689.5    ,  710.875  ,  731.8125 ,
        752.375  ,  772.59375,  792.4375 ,  811.9375 ,  831.125  ,
        850.     ,  868.5625 ,  886.84375,  904.84375,  922.59375,
        940.09375,  95

Reference waveform: {'plus': array([ 0.00000000e+00-0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
        0.00000000e+00-0.00000000e+00j, ...,
       -4.23008636e-25+7.98210938e-26j, -4.23011798e-25+7.97089719e-26j,
        0.00000000e+00-0.00000000e+00j]), 'cross': array([0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
       0.00000000e+00+0.00000000e+00j, ...,
       7.95519924e-26+4.21582544e-25j, 7.94402485e-26+4.21585695e-25j,
       0.00000000e+00+0.00000000e+00j])}


In [10]:
alt_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos,
    waveform_generator=waveform_generator,
)
print("Log Likelihood Test:", alt_likelihood.log_likelihood_ratio())


KeyError: 'a_2'